In [1]:
import boto3
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'ryan-greiner-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key = 'framingham.csv'
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading CSV file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [3]:
## Define input and target ##
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .2, stratify = Y)
X_train.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3348.000000,3377.000000,3389.000000
mean,49.599705,0.494690,237.237455,25.849769,75.915904
std,8.558958,0.500046,45.046682,4.154511,11.893405
min,32.000000,0.000000,107.000000,15.540000,44.000000
25%,42.000000,0.000000,206.000000,23.060000,68.000000
50%,49.000000,0.000000,234.000000,25.430000,75.000000
75%,56.750000,1.000000,264.000000,28.090000,82.000000
max,70.000000,1.000000,696.000000,56.800000,143.000000


In [4]:
X_test.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,848.000000,848.000000,840.000000,842.000000,848.000000
mean,49.525943,0.491745,234.665476,25.610451,75.731132
std,8.629555,0.500227,42.687112,3.763222,12.550945
min,33.000000,0.000000,124.000000,16.480000,45.000000
25%,42.000000,0.000000,206.000000,23.090000,67.000000
50%,49.000000,0.000000,232.500000,25.180000,75.000000
75%,56.000000,1.000000,261.250000,27.817500,84.000000
max,69.000000,1.000000,410.000000,43.690000,130.000000


In [5]:
## Define Imputer ##
imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')
imputer.fit(X_train)

## Impute Values ##
X_train_imp = pd.DataFrame(imputer.fit_transform(X_train), columns = X_train.columns)
X_test_imp = pd.DataFrame(imputer.fit_transform(X_test), columns = X_test.columns)


In [6]:
X_train_imp.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3390.000000,3390.000000,3390.000000
mean,49.599705,0.494690,237.197345,25.848159,75.915634
std,8.558958,0.500046,44.768112,4.146616,11.891660
min,32.000000,0.000000,107.000000,15.540000,44.000000
25%,42.000000,0.000000,206.000000,23.070000,68.000000
50%,49.000000,0.000000,234.000000,25.430000,75.000000
75%,56.750000,1.000000,263.000000,28.090000,82.000000
max,70.000000,1.000000,696.000000,56.800000,143.000000


In [7]:
X_test_imp.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,848.000000,848.000000,848.000000,848.000000,848.000000
mean,49.525943,0.491745,234.645047,25.607406,75.731132
std,8.629555,0.500227,42.485558,3.750043,12.550945
min,33.000000,0.000000,124.000000,16.480000,45.000000
25%,42.000000,0.000000,206.000000,23.090000,67.000000
50%,49.000000,0.000000,232.500000,25.180000,75.000000
75%,56.000000,1.000000,261.000000,27.780000,84.000000
max,69.000000,1.000000,410.000000,43.690000,130.000000


In [8]:
scaler = MinMaxScaler()

X_train_imp = pd.DataFrame(scaler.fit_transform(X_train_imp), columns = X_train_imp.columns)
X_test_imp = pd.DataFrame(scaler.fit_transform(X_test_imp), columns = X_test_imp.columns)

In [10]:
## Model ##
log_md = LogisticRegression().fit(X_train_imp, Y_train)

## Prediction ##
log_pred = log_md.predict_proba(X_test_imp)[:, 1]

## Label ##
log_label = np.where(log_pred < .1, 0, 1)

print(classification_report(Y_test, log_label))

              precision    recall  f1-score   support

           0       0.94      0.30      0.46       719
           1       0.19      0.90      0.31       129

    accuracy                           0.39       848
   macro avg       0.57      0.60      0.38       848
weighted avg       0.83      0.39      0.43       848



In [11]:
## Model ##
rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train_imp, Y_train)

## Prediction ##
rf_pred = rf_md.predict_proba(X_test_imp)[:, 1]

## Label ##
rf_label = np.where(rf_pred < .1, 0, 1)

print(classification_report(Y_test, rf_label))

              precision    recall  f1-score   support

           0       0.94      0.16      0.27       719
           1       0.17      0.95      0.29       129

    accuracy                           0.28       848
   macro avg       0.55      0.55      0.28       848
weighted avg       0.82      0.28      0.27       848



Based on my results, I would use logistic regression to predict Ten Year CHD